In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

VER = 175
USE = 'orders'

PART = 3

In [2]:
import pandas as pd, cudf, numpy as np, gc
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv(f"../../data/candidate_scores/submission_v711_LB.csv")
df = df.loc[df.session_type.str.contains(USE)]
df = cudf.DataFrame(df)
print( df.shape )
df.head()

(1671803, 2)


,session_type,labels
1671803,12899779_orders,59625 731692 737445 941596 1253524 1790770 258...
1671804,12899780_orders,1142000 736515 973453 582732 487136 1758603 15...
1671805,12899781_orders,918667 199008 194067 57315 141736 1681537 1628...
1671806,12899782_orders,834354 779477 595994 889671 740494 987399 9751...
1671807,12899783_orders,1817895 607638 1754419 1216820 1729553 300127 ...


In [4]:
df2 = pd.read_csv(f"../../data/candidate_scores/submission_v711_wgt_LB.csv")
df2 = df2.loc[df2.session_type.str.contains(USE)]
df2 = cudf.DataFrame(df2)
print( df2.shape )
df2.head()

(1671803, 2)


,session_type,labels
1671803,12899779_orders,-1 10.675 8.0 6.65 6.2 5.6 5.5 5.4 4.5 4.25 4....
1671804,12899780_orders,-1 -1 -1 -1 41.537499999999994 33.725 31.50000...
1671805,12899781_orders,-1 -1 -1 -1 -1 37.675 26.7 26.499999999999996 ...
1671806,12899782_orders,16.70052175932746 11.64797691294779 10.2636304...
1671807,12899783_orders,-1 -1 -1 -1 -1 -1 -1 -1 -1 17.7 13.4 11.5 11.4...


In [5]:
# FILTER USERS
UU = np.load(f'/raid/Kaggle/otto/data0/test_user_{PART}.npy')
print( len(UU) )

417950


In [6]:
df['user'] = df.session_type.str.partition('_')[0].astype('int32')
df = df.loc[df.user.isin(UU)]
df = df.drop('session_type',axis=1)
df.head()

,labels,user
1671803,59625 731692 737445 941596 1253524 1790770 258...,12899779
1671804,1142000 736515 973453 582732 487136 1758603 15...,12899780
1671805,918667 199008 194067 57315 141736 1681537 1628...,12899781
1671806,834354 779477 595994 889671 740494 987399 9751...,12899782
1671807,1817895 607638 1754419 1216820 1729553 300127 ...,12899783


In [7]:
df2['user'] = df2.session_type.str.partition('_')[0].astype('int32')
df2 = df2.loc[df2.user.isin(UU)]
df2 = df2.drop('session_type',axis=1)
df2.head()

,labels,user
1671803,-1 10.675 8.0 6.65 6.2 5.6 5.5 5.4 4.5 4.25 4....,12899779
1671804,-1 -1 -1 -1 41.537499999999994 33.725 31.50000...,12899780
1671805,-1 -1 -1 -1 -1 37.675 26.7 26.499999999999996 ...,12899781
1671806,16.70052175932746 11.64797691294779 10.2636304...,12899782
1671807,-1 -1 -1 -1 -1 -1 -1 -1 -1 17.7 13.4 11.5 11.4...,12899783


In [8]:
tmp = df.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
tmp['pos'] = np.tile( np.arange(100),tmp.shape[0]//100 ).astype('int32')
tmp.head()

,item,pos
1671803,59625,0
1671803,731692,1
1671803,737445,2
1671803,941596,3
1671803,1253524,4


In [9]:
users = df[['user']].copy()
users.head()

,user
1671803,12899779
1671804,12899780
1671805,12899781
1671806,12899782
1671807,12899783


In [10]:
users = users.merge(tmp, left_index=True, right_index=True)

In [11]:
del df,tmp
gc.collect()

58

In [12]:
users.head()

,user,item,pos
1671817,12899793,174443,88
1671817,12899793,122983,89
1671817,12899793,942510,90
1671817,12899793,1788508,91
1671817,12899793,1608131,92


In [13]:
users.dtypes

user    int32
item    int32
pos     int32
dtype: object

In [14]:
tmp = df2.labels.str.split(' ').explode().astype('float32').rename('wgt').to_frame()
tmp['pos'] = np.tile( np.arange(100),tmp.shape[0]//100 ).astype('int32')
tmp.head()

,wgt,pos
1671803,-1.000,0
1671803,10.675,1
1671803,8.000,2
1671803,6.650,3
1671803,6.200,4


In [15]:
users2 = df2[['user']].copy()#.reset_index()
users2.head()

,user
1671803,12899779
1671804,12899780
1671805,12899781
1671806,12899782
1671807,12899783


In [16]:
users2 = users2.merge(tmp, left_index=True, right_index=True)

In [17]:
users2.head()

,user,wgt,pos
1671901,12899877,1.0,88
1671901,12899877,1.0,89
1671901,12899877,1.0,90
1671901,12899877,1.0,91
1671901,12899877,1.0,92


In [18]:
users = users.merge(users2,on=['user','pos'],how='left')

In [19]:
users.head()

,user,item,pos,wgt
0,12899974,621705,36,6.3250
1,12899974,610276,37,5.9250
2,12899974,378616,38,5.9250
3,12899974,1283268,39,5.6500
4,12899974,659947,40,5.5875


In [20]:
del users2
gc.collect()

62

In [21]:
users.shape

(41795000, 4)

# Add Weights

In [22]:
for k in [1,3,4,5,6,7]:

    dfA = cudf.read_csv(f'../../data/candidate_scores/submission_v11_v{k}.csv')
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'../../data/candidate_scores/submission_v11_wgt_v{k}.csv')
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{k}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{k}']!=-2]
    del dfB
      
    #dfA = dfA.rename({'pos':f'pos_{k}'},axis=1)
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

1 (41795000, 5)
3 (41795000, 6)
4 (41795000, 7)
5 (41795000, 8)
6 (41795000, 9)
7 (41795000, 10)


In [23]:
#import gc
#users = users.to_pandas()
#gc.collect()

In [24]:
USE2 = 'orders'
OFFSET = 10
for j,k in enumerate(['../../data/candidate_scores/submission_v11_B',
                      '../../data/candidate_scores/submission_v13']):

    dfA = cudf.read_csv(f'{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'{k}_wgt.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    #dfA = dfA.rename({'pos':f'pos_{k}'},axis=1)
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

../../../Dec-2022/dec-6c-22-otto/submission_v11_B (41795000, 11)
../../../Dec-2022/dec-18-22-otto/submission_v13 (41795000, 12)


In [25]:
USE2 = 'clicks'
OFFSET = 12
for j,k in enumerate(['../../data/candidate_scores/submission_v11_B',
                      '../../data/candidate_scores/submission_v13']):

    dfA = cudf.read_csv(f'{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'{k}_wgt.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    #dfA = dfA.rename({'pos':f'pos_{k}'},axis=1)
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

../../../Dec-2022/dec-6c-22-otto/submission_v11_B (41795000, 13)
../../../Dec-2022/dec-18-22-otto/submission_v13 (41795000, 14)


In [26]:
#users = users.to_pandas()
#gc.collect()

In [27]:
if USE=='clicks': USE2 = 'orders'
else: USE2 = 'clicks'
    
OFFSET = 30
for j,k in enumerate([f"../../data/candidate_scores/submission_v711_LB"]):

    dfA = pd.read_csv(f'{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.loc[dfA.user.isin(UU)]
    dfA = dfA.drop('session_type',axis=1)
    dfA = cudf.DataFrame(dfA)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(100),tmpA.shape[0]//100 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = pd.read_csv(f'../../data/candidate_scores/submission_v711_wgt_LB.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.loc[dfB.user.isin(UU)]
    dfB = dfB.drop('session_type',axis=1)
    dfB = cudf.DataFrame(dfB)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(100),tmpB.shape[0]//100 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    #dfA = dfA.rename({'pos':f'pos_{k}'},axis=1)
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

/raid/Kaggle/otto/data0/submission_v711_LB (41795000, 15)


In [28]:
USE2 = 'orders'
OFFSET = 14

for j,k in enumerate( range(20,28) ):

    dfA = cudf.read_csv(f'../../data/candidate_scores/submission_v11_v{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'../../data/candidate_scores/submission_v11_wgt_v{k}.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    #dfA = dfA.rename({'pos':f'pos_{k}'},axis=1)
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

20 (41795000, 16)
21 (41795000, 17)
22 (41795000, 18)
23 (41795000, 19)
24 (41795000, 20)
25 (41795000, 21)
26 (41795000, 22)
27 (41795000, 23)


In [29]:
USE2 = 'orders'
OFFSET = 22
for j,k in enumerate(['../../data/candidate_scores/submission_v14']):

    dfA = cudf.read_csv(f'{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'{k}_wgt.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

../../../Dec-2022/dec-18-22-otto/submission_v14 (41795000, 24)


In [30]:
USE2 = 'clicks'
OFFSET = 23
for j,k in enumerate(['../../data/candidate_scores/submission_v14']):

    dfA = cudf.read_csv(f'{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'{k}_wgt.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

../../../Dec-2022/dec-18-22-otto/submission_v14 (41795000, 25)


In [31]:
#USE2 = 'orders'
OFFSET = 24

for j,(k,USE2) in enumerate( zip( range(31,34),['orders','orders','clicks'] ) ):

    dfA = cudf.read_csv(f'../../data/candidate_scores/submission_v11_v{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'../../data/candidate_scores/submission_v11_wgt_v{k}.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    #dfA = dfA.rename({'pos':f'pos_{k}'},axis=1)
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

31 (41795000, 26)
32 (41795000, 27)
33 (41795000, 28)


In [32]:
users = users.to_pandas()
gc.collect()

0

In [33]:
users.head()

,user,item,pos,wgt,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26
0,12899886,1804439,84,2.00,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,1.0,-3.0
1,12899886,904709,85,2.00,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0
2,12899886,1822939,88,2.00,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.0,-3.0
3,12899886,723975,93,1.95,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,-3.0,-3.0
4,12899886,1798165,90,2.00,-3.0,-3.0,-3.0,-3.0,-3.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0


# User and Item Features

In [34]:
item_features = pd.read_parquet('../../data/item_user_features/item14b_LB.pqt')
for c in item_features.columns:
    if str(item_features[c].dtype)=='int32':
        print(c, item_features[c].dtype)
        item_features[c] = item_features[c].astype('float32')
print( item_features.shape )
item_features.head()

(1848283, 7)


,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5
aid,,,,,,,
0,0.0,20.0,17.0,1.176471,0.0,<NA>,<NA>
1,0.052632,19.0,17.0,1.117647,0.229416,<NA>,<NA>
2,0.0,11.0,10.0,1.1,0.0,<NA>,<NA>
3,0.104082,1470.0,715.0,2.055944,0.353024,<NA>,<NA>
4,0.0,64.0,51.0,1.254902,0.0,<NA>,<NA>


In [35]:
users = users.merge(item_features, left_on='item', right_index=True, how='left')
del item_features
gc.collect()
users.head()

,user,item,pos,wgt,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5
0,12899886,1804439,84,2.00,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,1.0,-3.0,0.076923,91.0,70.0,1.3,0.306622,<NA>,<NA>
1,12899886,904709,85,2.00,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,0.0,597.0,533.0,1.120075,0.0,<NA>,<NA>
2,12899886,1822939,88,2.00,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.0,-3.0,0.02381,42.0,34.0,1.235294,0.154303,<NA>,<NA>
3,12899886,723975,93,1.95,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.037838,370.0,266.0,1.390978,0.229706,<NA>,<NA>
4,12899886,1798165,90,2.00,-3.0,-3.0,-3.0,-3.0,-3.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.0,12.0,12.0,1.0,0.0,<NA>,<NA>


In [36]:
item_features = pd.read_parquet('../../data/item_user_features/item13_LB.pqt')
for c in item_features.columns:
    if str(item_features[c].dtype)=='int32':
        print(c, item_features[c].dtype)
        item_features[c] = item_features[c].astype('float32')
print( item_features.shape )
item_features.head()

count_item_0_6 int32
count_item_1_6 int32
count_item_2_6 int32
count_item_3_6 int32
count_item_4_6 int32
count_item_5_6 int32
count_item_6_6 int32
count_user_0_6 int32
count_user_1_6 int32
count_user_2_6 int32
count_user_3_6 int32
count_user_4_6 int32
count_user_5_6 int32
count_user_6_6 int32
(783486, 23)


,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6
5576,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.0,0.755929,0.755929
5579,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.377964,0.377964
5580,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,0.786796,0.786796
5582,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.377964,0.377964
5584,-1.0,-1.0,0.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,0.487950,0.487950


In [37]:
users = users.merge(item_features, left_on='item', right_index=True, how='left')
del item_features
gc.collect()
users.head()

,user,item,pos,wgt,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6
0,12899886,1804439,84,2.00,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,1.0,-3.0,0.076923,91.0,70.0,1.3,0.306622,<NA>,<NA>,0.00,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.755929,0.755929
1,12899886,904709,85,2.00,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,0.0,597.0,533.0,1.120075,0.0,<NA>,<NA>,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,5.0,3.0,4.0,3.0,7.0,3.0,10.0,5.0,3.0,4.0,3.0,6.0,3.0,10.0,2.645751,2.645751
2,12899886,1822939,88,2.00,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.0,-3.0,0.02381,42.0,34.0,1.235294,0.154303,<NA>,<NA>,-1.00,0.0,-1.000000,-1.0,-1.0,-1.0,0.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,0.487950,0.487950
3,12899886,723975,93,1.95,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.037838,370.0,266.0,1.390978,0.229706,<NA>,<NA>,0.25,0.0,0.142857,0.0,0.0,0.0,0.0,8.0,11.0,7.0,1.0,3.0,3.0,3.0,5.0,9.0,3.0,1.0,2.0,3.0,3.0,3.579040,3.579040
4,12899886,1798165,90,2.00,-3.0,-3.0,-3.0,-3.0,-3.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.0,12.0,12.0,1.0,0.0,<NA>,<NA>,-1.00,-1.0,-1.000000,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.511858,1.511858


In [38]:
item_features = pd.read_parquet('../../data/item_user_features/item4b_LB.pqt')
for c in item_features.columns:
    if str(item_features[c].dtype)=='int32':
        print(c, item_features[c].dtype)
        item_features[c] = item_features[c].astype('float32')
print( item_features.shape )
item_features.head()

count_item int32
count_user int32
(1839558, 9)


,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std
aid,,,,,,,,,
0,0.000000,33.0,29.0,50843.453125,388588.906250,1.137931,0.000000,21374.218750,148150.937500
1,0.032258,31.0,27.0,49398.097656,258430.359375,1.148148,0.179605,18655.564453,160308.437500
2,0.000000,16.0,15.0,44175.187500,292575.187500,1.066667,0.000000,10662.440430,133456.968750
3,0.103150,2540.0,1268.0,50379.726562,293184.125000,2.003155,0.349970,16550.375000,148045.750000
4,0.049080,163.0,103.0,54755.527344,299113.812500,1.582524,0.216700,18486.285156,161496.953125


In [39]:
#users = users.to_pandas()

In [40]:
users = users.merge(item_features, left_on='item', right_index=True, how='left')

In [41]:
users.head()

,user,item,pos,wgt,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std
0,12899886,1804439,84,2.00,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,1.0,-3.0,0.076923,91.0,70.0,1.3,0.306622,<NA>,<NA>,0.00,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.755929,0.755929,0.051282,195.0,144.0,51672.507812,319734.031250,1.354167,0.263669,16252.813477,163825.265625
1,12899886,904709,85,2.00,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,0.0,597.0,533.0,1.120075,0.0,<NA>,<NA>,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,5.0,3.0,4.0,3.0,7.0,3.0,10.0,5.0,3.0,4.0,3.0,6.0,3.0,10.0,2.645751,2.645751,0.000974,1027.0,889.0,47974.835938,311801.906250,1.155231,0.031204,20589.056641,174983.484375
2,12899886,1822939,88,2.00,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.0,-3.0,0.02381,42.0,34.0,1.235294,0.154303,<NA>,<NA>,-1.00,0.0,-1.000000,-1.0,-1.0,-1.0,0.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,0.487950,0.487950,0.013889,72.0,52.0,48974.515625,286574.500000,1.384615,0.117851,18368.900391,158510.375000
3,12899886,723975,93,1.95,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.037838,370.0,266.0,1.390978,0.229706,<NA>,<NA>,0.25,0.0,0.142857,0.0,0.0,0.0,0.0,8.0,11.0,7.0,1.0,3.0,3.0,3.0,5.0,9.0,3.0,1.0,2.0,3.0,3.0,3.579040,3.579040,0.037288,590.0,420.0,45309.707031,306999.187500,1.404762,0.222578,20719.945312,174047.687500
4,12899886,1798165,90,2.00,-3.0,-3.0,-3.0,-3.0,-3.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.0,12.0,12.0,1.0,0.0,<NA>,<NA>,-1.00,-1.0,-1.000000,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.511858,1.511858,0.000000,18.0,16.0,43315.332031,254515.328125,1.125000,0.000000,17750.130859,141589.156250


In [42]:
del item_features
gc.collect()

0

In [43]:
item_features = pd.read_parquet('../../data/item_user_features/item10_LB.pqt')
for c in item_features.columns:
    if str(item_features[c].dtype)=='int32':
        print(c, item_features[c].dtype)
        item_features[c] = item_features[c].astype('float32')
print( item_features.shape )
users = users.merge(item_features, left_on='item', right_index=True, how='left')
users.head()

count_item3 int32
count_user3 int32
prev3 int32
next3 int32
orders3 int32
carts3 int32
(783486, 15)


,user,item,pos,wgt,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3
0,12899886,1804439,84,2.00,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,1.0,-3.0,0.076923,91.0,70.0,1.3,0.306622,<NA>,<NA>,0.00,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.755929,0.755929,0.051282,195.0,144.0,51672.507812,319734.031250,1.354167,0.263669,16252.813477,163825.265625,0.000000,2.0,2.0,61511.000000,407111.00000,1.000000,0.000000,7998.791992,7998.791992,2.0,2.0,0.0,0.0,-1.0,-1.0
1,12899886,904709,85,2.00,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,0.0,597.0,533.0,1.120075,0.0,<NA>,<NA>,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,5.0,3.0,4.0,3.0,7.0,3.0,10.0,5.0,3.0,4.0,3.0,6.0,3.0,10.0,2.645751,2.645751,0.000974,1027.0,889.0,47974.835938,311801.906250,1.155231,0.031204,20589.056641,174983.484375,0.000000,35.0,34.0,51473.769531,295862.34375,1.029412,0.000000,19999.134766,163529.546875,13.0,13.0,0.0,0.0,-1.0,-1.0
2,12899886,1822939,88,2.00,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.0,-3.0,0.02381,42.0,34.0,1.235294,0.154303,<NA>,<NA>,-1.00,0.0,-1.000000,-1.0,-1.0,-1.0,0.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,0.487950,0.487950,0.013889,72.0,52.0,48974.515625,286574.500000,1.384615,0.117851,18368.900391,158510.375000,0.000000,2.0,2.0,52410.500000,398010.50000,1.000000,0.000000,6465.077148,115722.976562,2.0,1.0,0.0,0.0,-1.0,-1.0
3,12899886,723975,93,1.95,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.037838,370.0,266.0,1.390978,0.229706,<NA>,<NA>,0.25,0.0,0.142857,0.0,0.0,0.0,0.0,8.0,11.0,7.0,1.0,3.0,3.0,3.0,5.0,9.0,3.0,1.0,2.0,3.0,3.0,3.579040,3.579040,0.037288,590.0,420.0,45309.707031,306999.187500,1.404762,0.222578,20719.945312,174047.687500,0.083333,36.0,26.0,48129.332031,400929.34375,1.384615,0.280306,20379.363281,146159.843750,18.0,19.0,0.0,3.0,-1.0,1.0
4,12899886,1798165,90,2.00,-3.0,-3.0,-3.0,-3.0,-3.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.0,12.0,12.0,1.0,0.0,<NA>,<NA>,-1.00,-1.0,-1.000000,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.511858,1.511858,0.000000,18.0,16.0,43315.332031,254515.328125,1.125000,0.000000,17750.130859,141589.156250,0.000000,4.0,1.0,80057.500000,598457.50000,4.000000,0.000000,188.468384,188.468384,4.0,4.0,0.0,0.0,-1.0,-1.0


In [44]:
del item_features
gc.collect()

0

In [45]:
user_features = pd.read_parquet('../../data/item_user_features/user4_LB.pqt')
for c in user_features.columns:
    if str(user_features[c].dtype)=='int32':
        print(c, user_features[c].dtype)
        user_features[c] = user_features[c].astype('float32')
print( user_features.shape )
user_features.head()

count_item2 int32
unique_item2 int32
(1671803, 9)


,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2,repeat2,buy_ratio_std2,hour_std2,day_std2
session,,,,,,,,,
12899779,0.000000,1.0,1.0,79200.000000,338400.00000,1.000000,-1.000000,-1.000000,-1.000000
12899780,0.000000,5.0,4.0,79291.601562,338491.59375,1.250000,0.000000,62.875275,62.875275
12899781,0.090909,11.0,5.0,68086.093750,288013.37500,2.200000,0.301511,12359.510742,143144.843750
12899782,0.457143,70.0,38.0,64777.812500,398034.96875,1.842105,0.695445,14570.271484,27593.218750
12899783,0.000000,11.0,9.0,70573.726562,329773.71875,1.222222,0.000000,17300.041016,95597.734375


In [46]:
users = users.merge(user_features, left_on='user', right_index=True, how='left')

In [47]:
del user_features
gc.collect()

0

In [48]:
users.head()

,user,item,pos,wgt,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2,repeat2,buy_ratio_std2,hour_std2,day_std2
0,12899886,1804439,84,2.00,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,1.0,-3.0,0.076923,91.0,70.0,1.3,0.306622,<NA>,<NA>,0.00,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.755929,0.755929,0.051282,195.0,144.0,51672.507812,319734.031250,1.354167,0.263669,16252.813477,163825.265625,0.000000,2.0,2.0,61511.000000,407111.00000,1.000000,0.000000,7998.791992,7998.791992,2.0,2.0,0.0,0.0,-1.0,-1.0,0.0,1.0,1.0,79222.0,338422.0,1.0,-1.0,-1.0,-1.0
1,12899886,904709,85,2.00,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,0.0,597.0,533.0,1.120075,0.0,<NA>,<NA>,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,5.0,3.0,4.0,3.0,7.0,3.0,10.0,5.0,3.0,4.0,3.0,6.0,3.0,10.0,2.645751,2.645751,0.000974,1027.0,889.0,47974.835938,311801.906250,1.155231,0.031204,20589.056641,174983.484375,0.000000,35.0,34.0,51473.769531,295862.34375,1.029412,0.000000,19999.134766,163529.546875,13.0,13.0,0.0,0.0,-1.0,-1.0,0.0,1.0,1.0,79222.0,338422.0,1.0,-1.0,-1.0,-1.0
2,12899886,1822939,88,2.00,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.0,-3.0,0.02381,42.0,34.0,1.235294,0.154303,<NA>,<NA>,-1.00,0.0,-1.000000,-1.0,-1.0,-1.0,0.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,0.487950,0.487950,0.013889,72.0,52.0,48974.515625,286574.500000,1.384615,0.117851,18368.900391,158510.375000,0.000000,2.0,2.0,52410.500000,398010.50000,1.000000,0.000000,6465.077148,115722.976562,2.0,1.0,0.0,0.0,-1.0,-1.0,0.0,1.0,1.0,79222.0,338422.0,1.0,-1.0,-1.0,-1.0
3,12899886,723975,93,1.95,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.037838,370.0,266.0,1.390978,0.229706,<NA>,<NA>,0.25,0.0,0.142857,0.0,0.0,0.0,0.0,8.0,11.0,7.0,1.0,3.0,3.0,3.0,5.0,9.0,3.0,1.0,2.0,3.0,3.0,3.579040,3.579040,0.037288,590.0,420.0,45309.707031,306999.187500,1.404762,0.222578,20719.945312,174047.687500,0.083333,36.0,26.0,48129.332031,400929.34375,1.384615,0.280306,20379.363281,146159.843750,18.0,19.0,0.0,3.0,-1.0,1.0,0.0,1.0,1.0,79222.0,338422.0,1.0,-1.0,-1.0,-1.0
4,12899886,1798165,90,2.00,-3.0,-3.0,-3.0,-3.0,-3.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.0,12.0,12.0,1.0,0.0,<NA>,<NA>,-1.00,-1.0,-1.000000,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.511858,1.511858,0.000000,18.0,16.0,43315.332031,254515.328125,1.125000,0.000000,17750.130859,141589.156250,0.000000,4.0,1.0,80057.500000,598457.50000,4.000000,0.000000,188.468384,188.468384,4.0,4.0,0.0,0.0,-1.0,-1.0,0.0,1.0,1.0,79222.0,338422.0,1.0,-1.0,-1.0,-1.0


In [49]:
for c in users.columns:
    print(c,users[c].dtype)

user int32
item int32
pos int32
wgt float32
score_1 float32
score_3 float32
score_4 float32
score_5 float32
score_6 float32
score_7 float32
score_10 float32
score_11 float32
score_12 float32
score_13 float32
score_30 float32
score_14 float32
score_15 float32
score_16 float32
score_17 float32
score_18 float32
score_19 float32
score_20 float32
score_21 float32
score_22 float32
score_23 float32
score_24 float32
score_25 float32
score_26 float32
buy_ratio_5 Float32
count_item_5 Float32
count_user_5 Float32
repeat_5 Float32
buy_ratio_std_5 Float32
item_day_std_5 Float32
user_day_std_5 Float32
buy_ratio_0_6 float32
buy_ratio_1_6 float32
buy_ratio_2_6 float32
buy_ratio_3_6 float32
buy_ratio_4_6 float32
buy_ratio_5_6 float32
buy_ratio_6_6 float32
count_item_0_6 float32
count_item_1_6 float32
count_item_2_6 float32
count_item_3_6 float32
count_item_4_6 float32
count_item_5_6 float32
count_item_6_6 float32
count_user_0_6 float32
count_user_1_6 float32
count_user_2_6 float32
count_user_3_6 float3

In [50]:
for c in users.columns:
    t = str(users[c].dtype)
    if (t=='int64'): 
        users[c] = users[c].astype('int32')
        print('convert',c,'to int32')
    if (t=='float64')|(t=='Int32')|(t=='Float32'):
        users[c] = users[c].astype('float32')
        print('convert',c,'to float32')

convert buy_ratio_5 to float32
convert count_item_5 to float32
convert count_user_5 to float32
convert repeat_5 to float32
convert buy_ratio_std_5 to float32
convert item_day_std_5 to float32
convert user_day_std_5 to float32


In [51]:
#%%time
#users = users.sort_values(['user','pos']).reset_index(drop=True)

In [52]:
#users = users.reset_index(drop=True)
#users = users.to_pandas()
#gc.collect()

In [53]:
users.head()

,user,item,pos,wgt,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2,repeat2,buy_ratio_std2,hour_std2,day_std2
0,12899886,1804439,84,2.00,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,1.0,-3.0,0.076923,91.0,70.0,1.300000,0.306622,NaN,NaN,0.00,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.755929,0.755929,0.051282,195.0,144.0,51672.507812,319734.031250,1.354167,0.263669,16252.813477,163825.265625,0.000000,2.0,2.0,61511.000000,407111.00000,1.000000,0.000000,7998.791992,7998.791992,2.0,2.0,0.0,0.0,-1.0,-1.0,0.0,1.0,1.0,79222.0,338422.0,1.0,-1.0,-1.0,-1.0
1,12899886,904709,85,2.00,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,0.000000,597.0,533.0,1.120075,0.000000,NaN,NaN,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,5.0,3.0,4.0,3.0,7.0,3.0,10.0,5.0,3.0,4.0,3.0,6.0,3.0,10.0,2.645751,2.645751,0.000974,1027.0,889.0,47974.835938,311801.906250,1.155231,0.031204,20589.056641,174983.484375,0.000000,35.0,34.0,51473.769531,295862.34375,1.029412,0.000000,19999.134766,163529.546875,13.0,13.0,0.0,0.0,-1.0,-1.0,0.0,1.0,1.0,79222.0,338422.0,1.0,-1.0,-1.0,-1.0
2,12899886,1822939,88,2.00,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.0,-3.0,0.023810,42.0,34.0,1.235294,0.154303,NaN,NaN,-1.00,0.0,-1.000000,-1.0,-1.0,-1.0,0.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,0.487950,0.487950,0.013889,72.0,52.0,48974.515625,286574.500000,1.384615,0.117851,18368.900391,158510.375000,0.000000,2.0,2.0,52410.500000,398010.50000,1.000000,0.000000,6465.077148,115722.976562,2.0,1.0,0.0,0.0,-1.0,-1.0,0.0,1.0,1.0,79222.0,338422.0,1.0,-1.0,-1.0,-1.0
3,12899886,723975,93,1.95,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.037838,370.0,266.0,1.390978,0.229706,NaN,NaN,0.25,0.0,0.142857,0.0,0.0,0.0,0.0,8.0,11.0,7.0,1.0,3.0,3.0,3.0,5.0,9.0,3.0,1.0,2.0,3.0,3.0,3.579040,3.579040,0.037288,590.0,420.0,45309.707031,306999.187500,1.404762,0.222578,20719.945312,174047.687500,0.083333,36.0,26.0,48129.332031,400929.34375,1.384615,0.280306,20379.363281,146159.843750,18.0,19.0,0.0,3.0,-1.0,1.0,0.0,1.0,1.0,79222.0,338422.0,1.0,-1.0,-1.0,-1.0
4,12899886,1798165,90,2.00,-3.0,-3.0,-3.0,-3.0,-3.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.000000,12.0,12.0,1.000000,0.000000,NaN,NaN,-1.00,-1.0,-1.000000,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.511858,1.511858,0.000000,18.0,16.0,43315.332031,254515.328125,1.125000,0.000000,17750.130859,141589.156250,0.000000,4.0,1.0,80057.500000,598457.50000,4.000000,0.000000,188.468384,188.468384,4.0,4.0,0.0,0.0,-1.0,-1.0,0.0,1.0,1.0,79222.0,338422.0,1.0,-1.0,-1.0,-1.0


In [54]:
users.to_parquet(f'../../data/infer_with_features/train_v{VER}_LB_{PART}.pqt',index=False)